ziad amr salah 1190103
ramy ashraf 

spice Circuit simulation engine

inputfile shape

    Netlist Format:
    Simulation_time
    # of iterations
    -Component 1
    -Component2
    …
    -1 (indicates end of input)
    Each component has the following format:
    ------------------------------------------------------------------------------
    Component_Type | Node1 | Node2 | Value | Initial_Value
    ------------------------------------------------------------------------------

    - Component_Type stands for the type of the component. It can be one of the following

In [107]:
import numpy as np


In [102]:
def read_circuit_file(file_name):
    with open(file_name) as f:
        lines = f.readlines()
    
    Vsources = {} 
    Resistors = {}
    Isources = {}
    sim_params = {}
    sim_params_count = 0
    R_count = 0
    Vsrc_count = 0
    Isrc_count = 0
    
    for line in lines:
        if line.startswith('-'):
            break
        
        parts = line.split()
        #print(parts)
        if parts[0] == 'Vsrc':
            Vsources[str(Vsrc_count)] = (parts[1][1], parts[2][1], parts[3], parts[4]) # ex for V0 Taking only the 0, V1 -> 1, V2 -> 2
            Vsrc_count+=1
        elif parts[0] == 'Isrc':
            Isources[str(Isrc_count)] = (parts[1][1], parts[2][1], parts[3], parts[4])
            Isrc_count+=1
        elif parts[0] == 'R':
            Resistors[str(R_count)] = (parts[1][1], parts[2][1], parts[3], parts[4])
            R_count+=1
        elif sim_params_count == 0:
            sim_params["Simulation_time"] = parts[0]
            sim_params_count += 1
        else:
            sim_params["# of iterations"] = parts[0]
            
    return sim_params, Resistors, Vsources, Isources

#read_circuit_file("./testcases/1.txt")

assert read_circuit_file("./testcases/1.txt") == ({'Simulation_time': '0.1', '# of iterations': '1'},
    {'0': ('1', '2', '3', '0'), '1': ('2', '0', '3', '0')},
    {'0': ('1', '0', '10', '0')},
    {})


In [103]:
def calculate_nodes(Resistors, Vsources, Isources):
    # Get list of all nodes
    nodes = set()
    
    for c in Resistors.values():
        nodes.add(c[0])
        nodes.add(c[1])
    for c in Vsources.values():
        nodes.add(c[0])
        nodes.add(c[1])
    for c in Isources.values():
        nodes.add(c[0])
        nodes.add(c[1])
    nodes = list(nodes)
    return nodes


assert calculate_nodes(read_circuit_file("./testcases/1.txt")[1], read_circuit_file("./testcases/1.txt")[2], read_circuit_file("./testcases/1.txt")[3]) == ['0', '2', '1']

#calculate_nodes(read_circuit_file("./testcases/1.txt")[1], read_circuit_file("./testcases/1.txt")[2])

In [104]:


def generate_A_matrix(nodes, Resistors, Vsources):

    n = len(nodes)  
    m = len(Vsources) 
    
    G = np.zeros((n,n))
    B = np.zeros((n,m))
    C = np.zeros((m,n))
    D = np.zeros((m,m))

    # Build G matrix
    for i in range(n):
        for j in range(n):
            conductance = 0
            if i==j:
                for c in Resistors.values():
                    #print(c)
                    if c[0] == nodes[i] or c[1] == nodes[i]:
                        conductance += 1/int(c[2]) 
            else:               
                for c in Resistors.values():
                    if (c[0] == nodes[i] and c[1] == nodes[j]) or (c[1] == nodes[i] and c[0] == nodes[j]):
                        conductance -= 1/int(c[2]) 
                        
            G[i,j] = conductance

    # Build B matrix        
    for i, c in enumerate(Vsources.values()):
        B[int(c[0]),i] = 1 # node1 is c[0] which is the node connected to positive battery side and c[1] is the negative node 
        B[int(c[1]),i] = -1 

    # Build C matrix (transpose of B)
    C = B.transpose()
    
    # Combine into A matrix
    A = np.bmat([[G, B], 
                    [C, D]])
    
    return A

generate_A_matrix(['0', '1', '2'], 
    {'0': ('1', '2', '3', '0'), '1': ('2', '0', '3', '0')},
    {'0': ('1', '0', '10', '0')})


matrix([[ 0.33333333,  0.        , -0.33333333, -1.        ],
        [ 0.        ,  0.33333333, -0.33333333,  1.        ],
        [-0.33333333, -0.33333333,  0.66666667,  0.        ],
        [-1.        ,  1.        ,  0.        ,  0.        ]])

In [120]:

def get_z_matrix(nodes, Vsources, Isources):

    n = len(nodes)
    m = len(Vsources)
    
    i = np.zeros(n)
    e = np.zeros(m)
    
    for idx, source in Isources.items():
        i[nodes.index(source[0])] += float(source[2]) # adding current into the node its entering

    vsrc_idx = 0 
    for idx, source in Vsources.items():
        e[vsrc_idx] = float(source[2])
        vsrc_idx += 1

    z = np.concatenate([i, e])
    
    # Split z into i and e
    # i = z[:A.shape[0]] 
    # e = z[A.shape[0]:]
    
    return z

get_z_matrix(['0', '1', '2'], 
    {'0': ('1', '0', '10', '0')},
    {})

# get_z_matrix(['0', '1', '2'], 
#         {'0': ('1', '0', '10', '0')},
#         {'0': ('1', '0', '10', '0')})

array([ 0., 10.,  0., 10.])

In [119]:

def solve(A, z):

    # Solve Ax = z
    #x = np.linalg.solve(A, z)
    x = np.linalg.pinv(A) @ z

    # Split x into v and j
    v = x[:A.shape[0]]
    j = x[A.shape[0]:]

    return v, j


sim_params, Resistors, Vsources, Isources  = read_circuit_file("./testcases/1.txt")
nodes = calculate_nodes(Resistors, Vsources, Isources)
A = generate_A_matrix(nodes, Resistors, Vsources)
z = get_z_matrix(nodes, Vsources, Isources)

solve(A, z)

(matrix([[-6.66666667,  3.33333333,  3.33333333, -3.33333333]]),
 matrix([], shape=(0, 4), dtype=float64))